In [1]:
from clinvar_functions import *
import pandas as pd
import matplotlib.pyplot as plt
import pysam

In [2]:
# Path to ClinVar VCF file
vcf_file = "clinvar.vcf"

# Open the VCF file
vcf_reader = pysam.VariantFile(vcf_file)

In [ ]:
variants = []

for record in vcf_reader.fetch():
    if 'ORC1' in record.info.get("GENEINFO", []):
        try: 
            variants.append(
            {
            "CHROM": record.chrom,  # Chromosome
            "POS": record.pos,      # Position
            "ID": record.id,        # Variant ID (e.g., rs number)
            "REF": record.ref,      # Reference allele
            "ALT": ",".join(str(alt) for alt in record.alts),  # Alternate allele(s)
            "QUAL": record.qual,    # Quality score
            "FILTER": record.filter.keys(),  # Filter status
            "INFO": record.info,    # Additional annotation info
            "RESULT": record.info.get("MC", [""])[0],   # Consequence type
            'CLNDN': record.info.get("CLNDN", [""])[0]
        }
            )
        except:
            pass

# Convert to DataFrame
df = pd.DataFrame(variants)